# Introduction to using Model Navigator functionality

#### Requirements
- Docker

#### Starting JupyterLab notebook
Use `start_notebook.sh` script located in the same directory as notebook `export_demo.ipynb` to start JupyterLab. It will pull Docker image and start container with notebook within. Then use URL printed in console to access notebook.

### Model Navigator Export from src consists of 3 parts:
1. Export
  1. Tests all export / conversion / runtime paths
  2. Not everything must pass
    1. (e.g. if ONNX-TRT is not supported, you can open a bug on Microsoft ONNX, but there is no guarantee they will support it any time soon)
  3. Minimal deployment paths
    1. [Navigator action] Export TorchScript (trace, script) or ONNX (trace)
        1. [User action] At least one path must be ensured by user, adjust / refactor your model to support export to TorchScript or ONNX
    2. [Navigator action] Correctness
    3. [Navigator action] Conversion
    4. [Navigator action] Performance
2. [non blocking User action] Model verification (MN is unable to automatically verify accuracy, we need user input to do that, by verifying exported / converted models on their metric function and setting the flag. Action is described as non-blocking because we can work without it just cannot verify model automatically)
  1. For desired deployment paths, user is responsible for model accuracy verification and set verified flag True
    1. TorchScript or ONNX
    2. [optional] Torch-TRT or TRT if available
3. Model save for distribution
  1. Save model in base format for distribution with information about successful conversions and verified models

[Documentation](https://github.com/triton-inference-server/model_navigator/blob/main/README.md) 

### Install Model Navigator Export API for PyTorch

In [ ]:
!cd /model_navigator && pip install --extra-index-url https://pypi.ngc.nvidia.com .[pyt,huggingface]

# Deep Learning Examples, PyTorch, ResNet50 example

### Clone Deep Learning Example

In [ ]:
!git clone https://github.com/NVIDIA/DeepLearningExamples DeepLearningExamples

### Add DeepLearningExamples directory to path and then import models

In [ ]:
import sys
sys.path.append("./DeepLearningExamples/PyTorch/Classification/ConvNets/")
from image_classification import models as convnet_models

### Add Dataloader
dataloader is iterable that contains tensors. In this example it is random torch tensor.

In [ ]:
import torch

dataloader = [torch.randn(1, 3, 224, 224, device="cuda")]

### Import model_navigator and use export function to export all possible formats for ResNet50 PyTorch

Model Navigator export API will automatically perform following actions:
1. export model from source code
2. convert model
3. correctness test on source and converted model outputs
4. performance evaluation

All artifacts will be stored inside `navigator_workdir/resnet50_pyt.nav.workspace` directory.

In [ ]:
import model_navigator as nav

source_model = convnet_models.resnet50(pretrained=True).eval()

pkg_desc = nav.torch.export(
   model=source_model,
   model_name="resnet50_pyt",
   dataloader=dataloader,
   target_device="cuda")

### Get ONNX Polygraphy runner for exported ONNX model and verify outputs

After exporting models, user have to manually verify them. It can be done by accessing model with `PackageDescriptor` and `get_model` function.

Loaded model can be used for inference, then outputs can be used for calculating custom metrics.

If model accuracy is valid then selected format can be set as verified.

In [ ]:
import numpy

feed_dict = {"input__0": dataloader[0].detach().cpu().numpy()}

# Compare in framework model and exported model outputs
output_a = [source_model(dataloader[0]).detach().cpu().numpy()]

onnx_runner = pkg_desc.get_runner(format=nav.Format.ONNX, runtime=nav.RuntimeProvider.CUDA)
with onnx_runner:
    output_b = onnx_runner.infer(feed_dict)
    
for a, b in zip(output_a, output_b.values()):
     assert numpy.allclose(a, b, atol=0.01, rtol=0.01)
    
pkg_desc.set_verified(format=nav.Format.ONNX, runtime=nav.RuntimeProvider.CUDA)

### Get TorchScript model and verify outputs

In [ ]:
feed_dict = {"input__0": dataloader[0].detach().cpu().numpy()}
    
# Compare in framework model and exported model outputs
output_a = source_model(dataloader[0]).detach().cpu().numpy()

ts_runner = pkg_desc.get_runner(format=nav.Format.TORCHSCRIPT, jit_type=nav.JitType.SCRIPT)
with ts_runner:
    output_b = ts_runner.infer(feed_dict)

for a, b in zip(output_a, output_b.values()):
     assert numpy.allclose(a, b, atol=0, rtol=0)
            
pkg_desc.set_verified(format=nav.Format.TORCHSCRIPT, jit_type=nav.JitType.SCRIPT, runtime=nav.RuntimeProvider.PYT)

### Save navigator package
After verification of selected formats user have to save navigator package. It will contain base formats (`TorchScript, ONNX`) and all information and statuses obtained during execution. Navigator package can be used latter to create all other formats (`Torch-TRT, TRT`) and rerurn tests.

This process will log warnings only for formats and runtimes that were not verified in previous steps.

In [ ]:
nav.save(pkg_desc, "resnet50_pyt.nav")

# HuggingFace, PyTorch, distilbert-base-uncased example

In [ ]:
import model_navigator as nav

export_config = {
    "model_name": "distilbert-base-uncased",
    "dataset_name": "imdb",
    "padding": "max_length",
    "max_sequence_len": 384,
    "max_bs": 2,
    "sample_count": 10,
}

pkg_desc = nav.contrib.huggingface.torch.export(**export_config)

### Save distilbert without verification
Even if you do not have access to dataset or source model, you still can save Navigator package. This process will log warnings for all formats.

In [ ]:
nav.save(pkg_desc, "distilbert_base_uncased.nav")